In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("movie_metadata.csv")

In [3]:
data_use = data.ix[:,['genres','plot_keywords','movie_title','actor_1_name',
                      'actor_2_name','actor_3_name','director_name','imdb_score']]
#data_use['movie_title'] = [i.replace("\xa0","") for i in list(data_use['movie_title'])]

In [4]:
#data_use['movie_title']

In [5]:
data_use['movie_title'] = [i.replace("\xc2\xa0","") for i in list(data_use['movie_title'])]
#print data_use['movie_title']

In [6]:
data_use.shape

(5043, 8)

In [7]:
clean_data = data_use.dropna(axis=0)

In [8]:
print data_use.shape
print clean_data.shape

(5043, 8)
(4781, 8)


In [9]:
#clean_data

In [10]:
clean_data = clean_data.drop_duplicates(['movie_title'])


In [11]:
clean_data = clean_data.reset_index(drop=True)
print (clean_data.shape)

(4659, 8)


In [12]:
people_list = []
for i in range(clean_data.shape[0]):
    name1 = clean_data.ix[i,'actor_1_name'].replace(" ","_")
    name2 = clean_data.ix[i,'actor_2_name'].replace(" ","_")
    name3 = clean_data.ix[i,'actor_3_name'].replace(" ","_")
    name4 = clean_data.ix[i,'director_name'].replace(" ","_")
    people_list.append("|".join([name1,name2,name3,name4]))
clean_data['people'] = people_list

In [13]:
#clean_data['people']

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

def token(text):
    return(text.split('|'))

cv_kw = CountVectorizer(max_features=100,tokenizer=token)
keywords = cv_kw.fit_transform(clean_data["plot_keywords"])
keywords_list = ["kw_" + i for i in cv_kw.get_feature_names()]
#print keywords
#print keywords_list
#clean_data.head()

cv_pp = CountVectorizer(max_features=100,tokenizer=token)
people = cv_pp.fit_transform(clean_data["people"])
people_list = ["pp_" + i for i in cv_pp.get_feature_names()]

cv_ge = CountVectorizer(tokenizer=token)
genres = cv_ge.fit_transform(clean_data["genres"])
genres_list = ["genres_" + i for i in cv_ge.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list
#print criterion_list

In [15]:
#print criterion_list

In [16]:
from sklearn.cluster import KMeans

mod = KMeans(n_clusters=100)
category = mod.fit_predict(cluster_data)
category_dataframe = pd.DataFrame({"category":category},index=clean_data['movie_title'])
#print category

In [17]:
#clean_data.ix[list(category_dataframe['category']==0),['genres','movie_title','people']]

In [18]:
cluster_data.shape

(4659L, 224L)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
def recommend(movie_name,recommend_number = 5):
    if movie_name in list(clean_data['movie_title']):
        movie_cluster = category_dataframe.ix[movie_name,'category']
        score = clean_data.ix[list(category_dataframe['category'] == movie_cluster),['imdb_score','movie_title']]
        sort_score = score.sort_values(['imdb_score'],ascending=[0])
        sort_score = sort_score[sort_score['movie_title'] != movie_name]
        recommend_number = min(sort_score.shape[0],recommend_number)
        recommend_movie = list(sort_score.iloc[range(recommend_number),1])
        print(recommend_movie)
    else:
        print("Can't find this movie!")

In [21]:
recommend('Robin Hood',10)

['Into the Wild', 'Dark City', 'A History of Violence', 'A.I. Artificial Intelligence', 'Race', "Marilyn Hotchkiss' Ballroom Dancing and Charm School", 'One True Thing', 'The Incredible Hulk', 'Tuck Everlasting', 'Vantage Point']


In [22]:
#any(clean_data.genres=='Comedy')

In [23]:
#a = clean_data.ix[4466]['movie_title']
#print a+"1"